<a href="https://colab.research.google.com/github/VineetMakharia/NN-ipynbs/blob/main/Copy_of_NN_IOT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
 from google.colab import files 
  
  
uploaded = files.upload()

Saving 9.csv to 9.csv


In [110]:
import pandas as pd 
import io 
  
df = pd.read_csv(io.BytesIO(uploaded['9.csv']), names= ['x1', 'x2', 'x3', 'x4', 'x5', 'y']) 

In [111]:
X = df[['x1','x2','x3','x4','x5']]
X.head()

,x1,x2,x3,x4,x5
0,43.060,80.829,154.37,205.63,253.82
1,54.100,79.580,118.67,207.98,257.62
2,49.086,76.696,136.36,212.10,243.83
3,35.767,91.612,122.82,236.30,245.87
4,55.154,85.700,116.98,245.92,262.81


In [112]:
Y = df[['y']]
Y.head()

,y
0,5659.4
1,5517.6
2,5514.7
3,5662.9
4,5864.4


In [113]:
from sklearn.model_selection import train_test_split
# Total 2300 samples, need 300 for test, so testsize = 300/2300 ~= 0.13
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.13)
assert len(X_train) + len(X_test) == len(X)

In [114]:
# Need to use a scaler for a neural network, popular ones are Standard Scaler and Min Max Scaler
# It is important to scale our train and test separately else we might get skewed results.
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [115]:
X_train.shape

(2001, 5)

In [140]:
from sklearn.model_selection import GridSearchCV, KFold
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from keras.optimizers import Adam
from keras.layers import Dropout

### Creating a base model with some parameters to find the R2 value, we are looking to get closer to a value of 1. 

In [117]:
def base_model(optimizer='adam',unit = 256,kernel_initializer='normal'):
  model = Sequential()
  model.add(Dense(128, kernel_initializer='normal',input_dim = 5, activation='relu'))

  # Creating 3 hidden layers
  model.add(Dense(units=unit, kernel_initializer=kernel_initializer,activation='relu'))
  model.add(Dense(units=unit, kernel_initializer=kernel_initializer,activation='relu'))
  model.add(Dense(units=unit, kernel_initializer=kernel_initializer,activation='relu'))

  # The Output Layer :
  model.add(Dense(1, kernel_initializer=kernel_initializer,activation='linear'))

  model.compile(loss='mean_absolute_error', optimizer=optimizer, metrics=['mean_absolute_error'])
  model.summary()
  return model

In [118]:
model = base_model()
model.fit(X_train, y_train,
          batch_size=256,
          epochs=500)
y_pred = model.predict(X_test)

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_85 (Dense)             (None, 128)               768       
_________________________________________________________________
dense_86 (Dense)             (None, 256)               33024     
_________________________________________________________________
dense_87 (Dense)             (None, 256)               65792     
_________________________________________________________________
dense_88 (Dense)             (None, 256)               65792     
_________________________________________________________________
dense_89 (Dense)             (None, 1)                 257       
Total params: 165,633
Trainable params: 165,633
Non-trainable params: 0
_________________________________________________________________
Epoch 1/500
8/8 [==============================] - 0s 14ms/step - loss: 5566.8324 - mean_absolute_error: 5566.832

In [119]:
from sklearn.metrics import r2_score
r2_score(y_test,y_pred)

0.9969848925607093

In [120]:
from sklearn.metrics import mean_squared_error
test_loss = []
test_loss.append(mean_squared_error(y_test, y_pred))
print("Test loss: {}".format(test_loss))

Test loss: [140.653175148908]


# Now with our base model, we are getting close to 1, we can do the hyper   parameter tuning

In [122]:
def tuning_model(unit = 256,kernel_initializer='normal'):
  model = Sequential()
  model.add(Dense(128, kernel_initializer='normal',input_dim = 5, activation='relu'))

  # Creating 3 hidden layers
  model.add(Dense(units=unit, kernel_initializer=kernel_initializer,activation='relu'))
  model.add(Dense(units=unit, kernel_initializer=kernel_initializer,activation='relu'))
  model.add(Dense(units=unit, kernel_initializer=kernel_initializer,activation='relu'))

  # The Output Layer :
  model.add(Dense(1, kernel_initializer=kernel_initializer,activation='linear'))
  model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
  model.summary()
  return model

In [141]:
# Need a Keras wrapper over the base_model 
classifier = KerasRegressor(build_fn=tuning_model)

In [142]:
from sklearn.model_selection import GridSearchCV
# parameters ={'nb_epoch':[500,1000,2000],
#             'kernel_initializer':['random_uniform', 'normal'],
#             'unit':[128,256,512],
#              'learn_rate':[0.01, 0.1, 0.2],
#              'batch_size' : [128,256,512]}
parameters ={'nb_epoch':[500,1000,2000],
            'kernel_initializer':['random_uniform', 'normal'],
            'unit':[128,256,512],
             'batch_size' : [128,256,512]}
gridSearch = GridSearchCV(classifier,param_grid=parameters,
                          cv=4,
                         n_jobs=-1,
                         return_train_score=True,
                          scoring='r2',
                          verbose = 2
                          )

In [143]:
import tensorflow as tf
tf.keras.backend.set_floatx('float64')
hist = gridSearch.fit(X_train, y_train)

Fitting 4 folds for each of 54 candidates, totalling 216 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   26.4s
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 216 out of 216 | elapsed:  2.1min finished


Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_105 (Dense)            (None, 128)               768       
_________________________________________________________________
dense_106 (Dense)            (None, 512)               66048     
_________________________________________________________________
dense_107 (Dense)            (None, 512)               262656    
_________________________________________________________________
dense_108 (Dense)            (None, 512)               262656    
_________________________________________________________________
dense_109 (Dense)            (None, 1)                 513       
Total params: 592,641
Trainable params: 592,641
Non-trainable params: 0
_________________________________________________________________
16/16 [==============================] - 0s 26ms/step - loss: 5558.4677 - mean_absolute_error: 5558.4676


In [144]:
print('Grid Search Best score',hist.best_score_)
print('Grid Search Best Parameters', hist.best_params_)
print('Execution time',hist.refit_time_)

Grid Search Best score -630.0527045677327
Grid Search Best Parameters {'batch_size': 128, 'kernel_initializer': 'normal', 'nb_epoch': 2000, 'unit': 512}
Execution time 1.2632217407226562


In [150]:
model1 = KerasRegressor(build_fn = base_model,verbose = 0,epochs = 2000,kernel_initializer='random_uniform',unit = 512, batch_size = 128)

In [151]:
model1.fit(X_train,y_train)

Model: "sequential_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_115 (Dense)            (None, 128)               768       
_________________________________________________________________
dense_116 (Dense)            (None, 512)               66048     
_________________________________________________________________
dense_117 (Dense)            (None, 512)               262656    
_________________________________________________________________
dense_118 (Dense)            (None, 512)               262656    
_________________________________________________________________
dense_119 (Dense)            (None, 1)                 513       
Total params: 592,641
Trainable params: 592,641
Non-trainable params: 0
_________________________________________________________________


In [160]:
y_pred1 = model1.predict(X_test)
print(y_test.head())
print(y_pred[:5])

           y
32    5655.7
1495  5562.2
262   5588.5
1386  5434.2
469   5514.3
[[5642.36083513]
 [5552.89922265]
 [5575.7465957 ]
 [5418.53345701]
 [5499.01500222]]


In [153]:
from sklearn.metrics import mean_squared_error
test_loss = []
test_loss.append(mean_squared_error(y_test, y_pred1))
print("Test loss: {}".format(test_loss))

Test loss: [965.1059368591498]


In [154]:
from sklearn.metrics import r2_score
print(r2_score(y_test,y_pred1))

0.9793115364310337
